<a href="https://colab.research.google.com/github/jetafese/btor2mlir/blob/ebpf/utils/ebpf/results/ebpf_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#eBPF Experiment Results


## Setup & Utilities

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Do not limit number of rows displayed
pd.set_option('display.max_rows', None)

/tmp/ipykernel_123703/1105105565.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [228]:
TIMEOUTMAX = 20
MEMOUTMAX = 200000

def clean_csv(in_file, ref=False):
  df = pd.read_csv(in_file)
  # drop rows that don't belong
  df = df[df["File"].str.contains("/ebpf-samples") == True]
  assert (df.count()['File'] == 369)
  # get program name, ignore full path
  df[['path', 'program']] = df['File'].str.split(pat = '/ebpf-samples', expand = True)
  df = df.drop(columns=['path', 'File'])
  # distinguish between reference results and current results
  if ref:
    df = df.rename(columns = {'Result': 'prevail'})
    df = df.rename(columns = {'Cpu': 'prevail_t'})
    df = df.rename(columns = {'Mem': 'prevail_m'})
    df = df[['program', 'prevail', 'prevail_t', 'prevail_m']]
    # handle NAs
    df['prevail_t'] = df['prevail_t'].fillna(TIMEOUTMAX)
    df['prevail_m'] = df['prevail_m'].fillna(MEMOUTMAX)
  else:
    df = df.rename(columns = {'Result': 'result'})
    # extract seahorn time
    df[['title', 'sea_time']] = df['seahorn_time'].str.split(pat = 'seahorn_total', expand = True)
    df = df.drop(columns=['title', 'seahorn_time'])
    # extract total time
    df[['time', 'mem_total']] = df['total'].str.split(pat = 'user', expand = True)
    df = df.drop(columns=['total'])
    # extract total memory
    df[['extra_1', 'mem_s']] = df['mem_total'].str.split(pat = '0avgdata', expand = True)
    df[['mem', 'extra_2']] = df['mem_s'].str.split(pat = 'maxresident', expand = True)
    df = df.drop(columns=['extra_1', 'mem_s', 'extra_2'])
    df = df[['program', 'result', 'trivial', 'sea_time', 'time', 'mem']]
    # handle NAs
    df['trivial'] = df['trivial'].fillna(0)
  return df

## Collect CSVs

In [230]:
#include csv for reference, current results
CURRENT = "/home/jetafese/btor2mlir/utils/ebpf/results/prevail_0919241154.csv"
REFERENCE = "/home/jetafese/btor2mlir/utils/ebpf/results/ref_prevail_0425241000.csv"

current_results = clean_csv(CURRENT)
ref_results = clean_csv(REFERENCE, ref=True)
# current_results
# ref_results

## Interesting points

In [235]:
trivialForSeahorn = pd.merge(current_results, ref_results, how='inner', on='program')
trivialForSeahorn[trivialForSeahorn['trivial'] == 1.0]

,program,result,trivial,sea_time,time,mem,prevail,prevail_t,prevail_m
52,/build/badhelpercall.o:.text,1,1.0,0.01,0.37,89280,0,0.000580,4408.0
72,/build/ringbuf_uninit.o:.text,1,1.0,0.01,0.37,89248,0,0.000751,4448.0
74,/build/tail_call_bad.o:xdp_prog,1,1.0,0.01,0.36,89488,0,0.000349,4580.0
75,/build/tail_call_bad.o:xdp_prog/0,1,1.0,0.01,0.35,89160,1,0.000156,4252.0
76,/build/tail_call.o:xdp_prog,1,1.0,0.01,0.34,89356,1,0.000385,4332.0
77,/build/tail_call.o:xdp_prog/0,1,1.0,0.01,0.34,89572,1,0.000188,4400.0
212,/linux/task_fd_query_kern.o:kprobe/blk_start_r...,1,1.0,0.01,0.38,89364,1,0.000161,4444.0
213,/linux/task_fd_query_kern.o:kretprobe/blk_acco...,1,1.0,0.01,0.35,89464,1,0.000151,4424.0
234,/linux/test_overhead_kprobe_kern.o:kprobe/uran...,1,1.0,0.00,0.37,89676,1,0.000144,4392.0
235,/linux/test_overhead_raw_tp_kern.o:raw_tracepo...,1,1.0,0.01,0.33,89056,1,0.000157,4264.0


## Checking Disparities

### Soundness checks

In [183]:
mergedDF = pd.merge(current_results, ref_results, how='inner', on='program')
diff = mergedDF['result'] != mergedDF['prevail']
diff.sum()
# mergedDF[diff]

167

In [175]:
# resultAndNOTPrevail = mergedDF.query('result > prevail')
resultAndNOTPrevail = mergedDF[mergedDF['result'].str.startswith("1") & mergedDF['prevail'].str.startswith("0")]
print('total cases where result == 1 and prevail == 0...', resultAndNOTPrevail.count()['program'])
# resultAndNOTPrevail

total cases where result == 1 and prevail == 0... 14


In [176]:
prevailAndNotResult = mergedDF[mergedDF['result'].str.startswith("0") & mergedDF['prevail'].str.startswith("1")]
print('total cases where result == 0 and prevail == 1...', prevailAndNotResult.count()['program'])
# prevailAndNotResult

total cases where result == 0 and prevail == 1... 86


In [177]:
prevailAndErrorResult = mergedDF[mergedDF['result'].str.startswith("error") & mergedDF['prevail'].str.startswith("1")]
print('total cases where result == error and prevail == 1...', prevailAndErrorResult.count()['program'])
# prevailAndErrorResult

total cases where result == error and prevail == 1... 5


In [178]:
NotPrevailAndErrorResult = mergedDF[mergedDF['result'].str.startswith("error") & mergedDF['prevail'].str.startswith("0")]
print('total cases where result == error and prevail == 0...', NotPrevailAndErrorResult.count()['program'])
# NotPrevailAndErrorResult

total cases where result == error and prevail == 0... 34


In [184]:
ErrorResultAndErrorPrevail = mergedDF[mergedDF['result'].str.startswith("error") & mergedDF['prevail'].str.contains("error")]
print('total cases where result == error and prevail == error...', ErrorResultAndErrorPrevail.count()['program'])
# ErrorResultAndErrorPrevail

total cases where result == error and prevail == error... 7


In [185]:
resultAndErrorPrevail = mergedDF[mergedDF['result'].str.startswith("1") & mergedDF['prevail'].str.contains("error")]
print('total cases where result == 1 and prevail == error...', resultAndErrorPrevail.count()['program'])
# resultAndErrorPrevail

total cases where result == 1 and prevail == error... 8


In [186]:
NotResultAndErrorPrevail = mergedDF[mergedDF['result'].str.startswith("0") & mergedDF['prevail'].str.contains("error")]
print('total cases where result == 0 and prevail == error...', NotResultAndErrorPrevail.count()['program'])
# NotResultAndErrorPrevail

total cases where result == 0 and prevail == error... 13


## Failing Pipelines


In [174]:
failures = pd.merge(current_results, ref_results, how='inner', on='program')
print('total number of failures: ', failures[failures['result'].str.contains("error")].count()['result'])
print('failures due to llvm casting: ', failures[failures['result'].str.contains("llvm dialect")].count()['result'])
print('failures due to translation code: ', failures[failures['result'].str.contains("translation")].count()['result'])
print('failures due to seahorn timeout: ', failures[failures['result'].str.contains("timeout")].count()['result'])

total number of failures:  46
failures due to llvm casting:  36
failures due to translation code:  4
failures due to seahorn timeout:  6


### llvm casting bug (dialect conversion)

In [59]:
castingBug = pd.merge(current_results, ref_results, how='inner', on='program')
castingBug[castingBug['result'].str.contains("llvm")]

,program,result,sea_time,time,mem,prevail,prevail_t,prevail_t
31,/bpf_cilium_test/bpf_lxc_jit.o:2/10,error: to llvm dialect conversion failed,None,NaN,NaN,unmarshaling error at 21: bad instruction op 0x30,NaN,NaN
32,/bpf_cilium_test/bpf_lxc_jit.o:2/7,error: to llvm dialect conversion failed,None,NaN,NaN,1,13.984400,13.984400
35,/bpf_cilium_test/bpf_lxc_jit.o:1/0xdc06,error: to llvm dialect conversion failed,None,NaN,NaN,1,10.103400,10.103400
89,/cilium/bpf_lxc.o:2/10,error: to llvm dialect conversion failed,None,NaN,NaN,unmarshaling error at 22: bad instruction op 0x30,NaN,NaN
111,/cilium/bpf_xdp_dsr_linux.o:2/21,error: to llvm dialect conversion failed,None,NaN,NaN,0,0.480340,0.480340
112,/cilium/bpf_xdp_dsr_linux.o:2/16,error: to llvm dialect conversion failed,None,NaN,NaN,0,9.190760,9.190760
113,/cilium/bpf_xdp_dsr_linux.o:2/18,error: to llvm dialect conversion failed,None,NaN,NaN,0,0.623473,0.623473
114,/cilium/bpf_xdp_dsr_linux.o:2/24,error: to llvm dialect conversion failed,None,NaN,NaN,0,4.956300,4.956300
116,/cilium/bpf_xdp_dsr_linux.o:2/15,error: to llvm dialect conversion failed,None,NaN,NaN,0,14.822500,14.822500
120,/cilium/bpf_xdp_dsr_linux.o:2/10,error: to llvm dialect conversion failed,None,NaN,NaN,0,0.175166,0.175166


### many exits in program (translation)

In [60]:
translationBug = pd.merge(current_results, ref_results, how='inner', on='program')
translationBug[translationBug['result'].str.contains("translation")]

,program,result,sea_time,time,mem,prevail,prevail_t,prevail_t
54,/build/byteswap.o:.text,error: translation failed,None,NaN,NaN,1,0.003497,0.003497
71,/build/prog_array.o:.text,error: translation failed,None,NaN,NaN,1,0.000282,0.000282
82,/build/wronghelper.o:xdp,error: translation failed,None,NaN,NaN,unmarshaling error at 7: invalid helper functi...,NaN,NaN
167,/invalid/invalid-lddw.o:.text,error: translation failed,None,NaN,NaN,unmarshaling error at 1: incomplete lddw,NaN,NaN


### seahorn timeout 

In [61]:
timeouts = pd.merge(current_results, ref_results, how='inner', on='program')
timeouts[timeouts['result'].str.contains("timeout")]

,program,result,sea_time,time,mem,prevail,prevail_t,prevail_t
50,/bpf_cilium_test/bpf_overlay.o:from-overlay,timeout: seahorn,None,NaN,NaN,unmarshaling error at 239: bad instruction op ...,NaN,NaN
90,/cilium/bpf_lxc.o:2/7,timeout: seahorn,None,NaN,NaN,1,13.66020,13.66020
119,/cilium/bpf_xdp_dsr_linux.o:2/7,timeout: seahorn,None,NaN,NaN,0,4.08059,4.08059
143,/cilium/bpf_xdp_dsr_linux_v1.o:2/7,timeout: seahorn,None,NaN,NaN,0,4.10145,4.10145
154,/cilium/bpf_xdp_snat_linux.o:2/7,timeout: seahorn,None,NaN,NaN,0,5.19936,5.19936
164,/cilium/bpf_xdp_snat_linux_v1.o:2/7,timeout: seahorn,None,NaN,NaN,0,5.00680,5.00680
